In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"
#os.environ["CUDA_VISIBLE_DEVICES"]=""
os.environ["HDF5_USE_FILE_LOCKING"] = "FALSE"

import datetime

import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

%load_ext autoreload
%autoreload 2

from dataset import Data
from callback import Save

from transformations import hitnet_trafo

In [2]:
data = Data()

In [3]:
hits_input = tf.keras.Input(shape=(5,))
params_input = tf.keras.Input(shape=(len(data.labels),))
t = hitnet_trafo(labels=data.labels)

h = t(hits_input, params_input)
h = tf.keras.layers.Dense(32, activation="relu")(h)
h = tf.keras.layers.Dense(64, activation="relu")(h)
h = tf.keras.layers.Dropout(0.001)(h)
h = tf.keras.layers.Dense(128, activation="relu")(h)
h = tf.keras.layers.Dropout(0.001)(h)
h = tf.keras.layers.Dense(256, activation="relu")(h)
h = tf.keras.layers.Dropout(0.001)(h)
h = tf.keras.layers.Dense(128, activation="relu")(h)
h = tf.keras.layers.Dropout(0.001)(h)
h = tf.keras.layers.Dense(64, activation="relu")(h)
h = tf.keras.layers.Dense(32, activation="relu")(h)
outputs = tf.keras.layers.Dense(1, activation='sigmoid')(h)

hit_net = tf.keras.Model(inputs=[hits_input, params_input], outputs=outputs)
hit_net.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 5)]          0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 8)]          0                                            
__________________________________________________________________________________________________
hitnet_trafo (hitnet_trafo)     (None, 15)           0           input_1[0][0]                    
__________________________________________________________________________________________________
dense (Dense)                   (None, 32)           512         hitnet_trafo[0][0]               
______________________________________________________________________________________________

In [4]:
optimizer = tf.keras.optimizers.Adam(0.00001)
hit_net.compile(loss='binary_crossentropy', optimizer=optimizer)

In [5]:
train_id = 'HitNet_' + datetime.datetime.now().strftime("%d_%b_%Y-%Hh%M")

In [6]:
callbacks = []
callbacks.append(Save(save_every=25, path_template='models/'+train_id+'/epoch_%i'))
callbacks.append(tf.keras.callbacks.TensorBoard(log_dir='logs/'+train_id, histogram_freq=1))

In [7]:
train_data, test_data = data.get_hitnet_data(train_batch_size=1024,
                                             test_batch_size=1024,
                                            )

In [ ]:
hist = hit_net.fit(train_data, epochs=10000, verbose=2, callbacks=callbacks, validation_data=test_data)

Epoch 1/10000
20997/20997 - 151s - loss: 0.4027 - val_loss: 0.0000e+00
Epoch 2/10000
20997/20997 - 74s - loss: 0.3731 - val_loss: 0.3638
Epoch 3/10000
20997/20997 - 74s - loss: 0.3650 - val_loss: 0.3570
Epoch 4/10000
20997/20997 - 74s - loss: 0.3594 - val_loss: 0.3518
Epoch 5/10000
20997/20997 - 74s - loss: 0.3539 - val_loss: 0.3440
Epoch 6/10000
20997/20997 - 74s - loss: 0.3483 - val_loss: 0.3402
Epoch 7/10000
20997/20997 - 72s - loss: 0.3418 - val_loss: 0.3316
Epoch 8/10000
20997/20997 - 71s - loss: 0.3351 - val_loss: 0.3254
Epoch 9/10000
20997/20997 - 70s - loss: 0.3294 - val_loss: 0.3194
Epoch 10/10000
20997/20997 - 70s - loss: 0.3248 - val_loss: 0.3169
Epoch 11/10000
20997/20997 - 70s - loss: 0.3210 - val_loss: 0.3130
Epoch 12/10000
20997/20997 - 69s - loss: 0.3178 - val_loss: 0.3105
Epoch 13/10000
20997/20997 - 69s - loss: 0.3150 - val_loss: 0.3062
Epoch 14/10000
20997/20997 - 67s - loss: 0.3127 - val_loss: 0.3061
Epoch 15/10000
20997/20997 - 71s - loss: 0.3107 - val_loss: 0.3041